# Logistic Regression Practice_01

### 타이타닉 승무원 데이터를 통한 생존 가능성 예측 

In [1]:
#from sklearn.linear_model import LogisticRegression

## Logistic Regression 모델 생성
아래와 같이 모델을 생성하고, 그 안에 속성들(features)과 그 레이블(labels)을 fit 시킨다.
fit() 메서드는 모델에 필요한 두 가지 변수를 전달해준다. 
#### 계수: model.coef_
#### 절편: model.intercept_

In [2]:
#model = LogisticRegression()
#model.fit(features, labels)

#### .predict()
를 사용하면 새로운 속성들을 넣엇을 때 그 레이블에 속하는지 아닌지 1 또는 0으로 구성된 벡터를 반환해준다.

그런데 만약 분류 결과가 아니라, 해당 레이블로 분류될 확률 값을 알고 싶다면 
#### .predict_proba()
를 사용해주면 된다. 각 샘플에 대한 확률을 0에서 1사이의 값으로 돌려준다.

In [3]:
#model.predict(features)

### 실제 로지스틱 회귀 알고리즘을 사용하기 전에는
### 반드시 데이터를 정규화(Normalization) 해줘야 한다.

## Sklearn LogisticRegression 실전 예제

### 1. Load Datas (데이터 불러오기)
pandas를 이용해 타이타닉 데이터를 불러온다.

891명의 데이터, 총 12개의 컬럼 확인

survived는 살았는지 죽었는지 확인하는 레이블 - 0 또는 1인데, 1이 살아남은 거다.

In [4]:
import pandas as pd

passengers = pd.read_csv("passengers.csv")
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

### 2. 데이터 전처리하기
#### 분석에 사용할 feature 고르기
일단 생존 여부에 중요한 영향을 미쳤을 것으로 예상하는 값들만 추려보면 Sex, Age, Pcalss를 생각해볼 수 있다.

여성, 어린이, 1/2/3등석 순으로 살아남을 확률이 높다고 가설을 세워본 거다.

#### 문자열을 숫자로 변환하기
머신러닝 라이브러리 sklearn에 넣어주기 전에 성별(Sex)는 숫자 데이터 1과 0으로 바꾼다. 여성이 살아남을 확률이 높다 가정하고 남성을 0, 여성을 1로 놓는다.

In [5]:
passengers['Sex'] = passengers['Sex'].map({'female': 1, 'male':0})

#### 결측치 채워주기
데이터를 확인하면 Age 값이 비어있는 경우가 있다. 해당 경우는 평균 값으로 대체해서 넣는다.

In [6]:
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

#### feature 분리하기
Pcalss의 경우 1등석에 탔는지, 2등석에 탔는지에 대해 각각의 feature로 만들어 주기
위해 컬럼을 새로 생성하자.

In [7]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x==1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x==2 else 0)

데이터 세트를 이렇게 준비할 수 있다.

In [8]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

### 3. 학습세트/평가세트 분리하기
sklearn의 train_test_split을 사용해서 학습세트와 평가 세트를 분리하자.

In [9]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

### 4. 데이터 정규화(스케일링) 하기
중요한 파트이다.
로지스틱 회귀는 실행할 때 regularation을 사용하기 때문에 실행 전에 우리가 가진 데이터를 스케일링하는 작업이 반드시 필요하다.

sklearn이 제공하는 StandardScaler를 활용해서 손쉽게 할 수 있다.
StandardScaler는 평균 0, 표준편차 1로 변환하는 방법이지만
이외에도 최소값0, 최대값 1이 되도록 변환하는 MinMaxscaler, 
중앙값(median) 0, IQR(Interquartile range) 1이 되도록 변환하는 RobustScaler 등이 있다.

StandardScaler를 사용하면 아래와 같이 데이터를 정규화(스케일링) 할 수 있다.


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

fit_transform은 fit과 transform을 합친 건데, fit은 각 속성(feature)마다 컬럼을
만드는 작업이라고 생각하면 된다. 이후 transform을 통해 데이터를 변형시키는 것이다.

위 코드에서도 학습세트로 fit을 한 번 해주었기 때문에, 평가세트에서는 별도로 fit을 할 필요 없이 바로 transform하면 되는 거다.

### 5. 모델 생성 및 평가하기
아래와 같이 데이터만 넣어주고 fit 해주면 끝이다.
학습 세트로 정확도를 바로 얻을 수 있다.



In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_features, train_labels)
print(model.score(train_features, train_labels))

0.7889221556886228


위의 값을 확인했을 경우 79% 정도 맞춘다고 한다.
이는 아까 분리해놓은 테스트 세트에서도 확인해볼 수 있다. 

In [12]:
print(model.score(test_features, test_labels))

0.8295964125560538


테스트 세트에서의 경우 80% 정도 맞출 수 있다.

이번에는 각 feature들의 계수(coefficients)를 확인해볼 차례이다. 어떤 feature가 생존에 큰 영향을 주는지 확인해볼 수 있다.


In [13]:
print(model.coef_)

[[ 1.18489758 -0.46960954  0.99522989  0.45064481]]


이전 코드에서 Sex, Age, FirstClass, SecondClass 순으로 넣었기 때문에 그 순서대로 확인하면 된다.

성별이 1(여자)이고, 일등석 탑승 여부가 중요하다는 걸 알 수 있다. 

반면 나이에 대한 계수는 음수가 나오는데 나이가 많을 수록 생존 확률이 낮아진다는
의미로 이해하면 되겠다.

### 6. 예측하기
이번엔 새로운 데이터를 넣어서 예측해볼것이다.

예를 들어 아래와 같이 타이타닉 영화 실제 주인공이었던 Jack, Rose의 값을 임의로 만들고, 나의 정보도 넣어봤다.

In [14]:
import numpy as np

Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
Me = np.array([0.0, 23.0, 1.0, 0.0])

sample_passengers = np.array([Jack, Rose, Me])

그 다음 위에서 했던 것처럼 데이터 스케일링을 다시 해줘야한다.

그리고 예측을 해보자.

In [15]:
sample_passengers = scaler.transform(sample_passengers)
print(model.predict(sample_passengers))

[0 1 1]


결과를 확인했을 때에 Jack만 죽고 나와 Rose는 사는 것으로 나온다.

In [16]:
print(model.predict_proba(sample_passengers))

[[0.87951256 0.12048744]
 [0.05179967 0.94820033]
 [0.44951539 0.55048461]]


각각 살아 남을 확률이 12%, 94%, 53%임을 확인할 수 있다.